# **Libraries & Constants**

---



## **Hazm**

Found existing installation: keras 2.15.0
Uninstalling keras-2.15.0:
  Successfully uninstalled keras-2.15.0
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tensorflow-intel 2.13.1
Uninstalling tensorflow-intel-2.13.1:
  Successfully uninstalled tensorflow-intel-2.13.1
Found existing installation: tensorflow-estimator 2.13.0
Uninstalling tensorflow-estimator-2.13.0:
  Successfully uninstalled tensorflow-estimator-2.13.0
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.


In [2]:
import os
import shutil

# مسیر پوشه منابع در دسکتاپ
existing_resources_dir = r"C:\Users\ExoGame\Desktop\resources"
# مسیر مقصد در مسیر پروژه
target_resources_dir = "resources"

# بررسی وجود پوشه منابع روی دسکتاپ
if os.path.exists(existing_resources_dir):
    print("در حال کپی کردن فایل‌های منابع از دسکتاپ...")

    # اگر پوشه resources در مسیر پروژه وجود دارد، حذف شود
    if os.path.exists(target_resources_dir):
        shutil.rmtree(target_resources_dir)

    # کپی پوشه منابع به مسیر پروژه
    shutil.copytree(existing_resources_dir, target_resources_dir)

    print("فایل‌های منابع با موفقیت به پوشه 'resources' منتقل شدند.")
else:
    print(f"خطا: پوشه منابع در مسیر '{existing_resources_dir}' یافت نشد.")


خطا: پوشه منابع در مسیر 'C:\Users\ExoGame\Desktop\resources' یافت نشد.


## **imports**

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Softmax, BatchNormalization, Dropout, Flatten, LSTM, GRU, SimpleRNN, Concatenate, concatenate, MaxPooling1D, GlobalMaxPool1D, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
# from keras.layers.merge import concatenate
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax, Nadam, SGD
from tensorflow.keras import Model
from tensorflow.keras.optimizers.schedules import InverseTimeDecay, ExponentialDecay
from tensorflow.keras.utils import to_categorical, plot_model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import os
import string
import cv2 as cv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from copy import deepcopy
from __future__ import unicode_literals
import hazm

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

# **Helper functions**

In [ ]:
def encode6ClassToOneHot(arr):
  tmp = np.zeros((arr.shape[0], 6))
  tmp[np.where(arr==1)[0], :] = [1, 0, 0, 0, 0, 0]
  tmp[np.where(arr==2)[0], :] = [0, 1, 0, 0, 0, 0]
  tmp[np.where(arr==3)[0], :] = [0, 0, 1, 0, 0, 0]
  tmp[np.where(arr==4)[0], :] = [0, 0, 0, 1, 0, 0]
  tmp[np.where(arr==5)[0], :] = [0, 0, 0, 0, 1, 0]
  tmp[np.where(arr==6)[0], :] = [0, 0, 0, 0, 0, 1]
  return tmp

def argmaxKeepDimensions(arr):
  tmp = np.zeros_like(arr)
  tmp[np.arange(len(arr)), arr.argmax(1)] = 1
  return tmp

def convertOneHotToClassNumber(arr):
  tmp = np.zeros((arr.shape[0], 1))
  tmp[np.arange(len(arr)), 0] = arr.argmax(1) + 1
  return tmp

In [ ]:
# create correct input form for SC NN
def encodeSubjects(subjects):
  subjects_encoded = []
  for subject in subjects:
    subjects_encoded.append(encodeWord(subject))

  subjects_encoded = np.array(subjects_encoded)

  subjects_encoded_pad = pad_sequences(subjects_encoded, maxlen=max_len, padding='post')
  subjects_encoded_OH = to_categorical(subjects_encoded_pad, num_classes=len(char_index))
  return subjects_encoded_OH

In [ ]:
def showPlots(model):
    plt.plot(model.history['accuracy'],
              label='training accuracy', marker='.', color='green')
    plt.plot(model.history['val_accuracy'],
              label='test accuracy', marker='.', color='blue')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    plt.plot(model.history['loss'],
              label='training loss', marker='.', color='green')
    plt.plot(model.history['val_loss'],
              label='test loss', marker='.', color='blue')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# **Subject Classsification NN (RNN)**



## **Preprocessing**

In [ ]:
# read subject classification dataset
scdataset = pd.read_excel('Datasets/subject classification dataset.xlsx')

# replace nan values with 'NA'
scdataset = scdataset.replace(to_replace = np.nan, value = 'NA')

In [ ]:
# data augmentation

classes = scdataset['Class']
class1_indices = np.where(classes == 1)
class2_indices = np.where(classes == 2)
class3_indices = np.where(classes == 3)

dataset_words = scdataset['Word'].values.tolist()

def augmentData(dataset_words, class_indices, class_num):
  type1 = [[dataset_words[i] + 'ی', class_num] for i in class_indices[0]]
  type2 = [[dataset_words[i] + 'ها', class_num] for i in class_indices[0]]
  type3 = [[dataset_words[i] + 'ان', class_num] for i in class_indices[0]]
  return type1 + type2 + type3

words = np.array(augmentData(dataset_words, class1_indices, 1) + augmentData(dataset_words, class2_indices, 2) + augmentData(dataset_words, class3_indices, 3))

dataAug = pd.DataFrame(words, columns=['Word', 'Class'])

scdataset_aug = pd.concat([scdataset, dataAug], axis=0, ignore_index=True)
scdataset_aug.style

In [ ]:
X_train_sc = np.array(scdataset['Word'])
y_train_sc = np.array(scdataset['Class'])

# one hot encoding for y_true
y_train_sc = encode6ClassToOneHot(y_train_sc)  

print(X_train_sc.shape)
print(y_train_sc.shape)

In [ ]:
# generate character encoding
puncs = list(string.punctuation) + [' ', '1', '\xa0', 'ِ', '\u200c', 'ُ', 'َ', '٫', 'ّ', 'ْ', '۰', '۳', '٫']
oov_token = 'not available char'
char_index = {oov_token: 0}

# encode input with encoding character set
for i in range(len(X_train_sc)):
    chars = list(X_train_sc[i])
    for c in chars:
        if c not in char_index and c not in puncs:
            char_index[c] = len(char_index)

len(char_index)  
print(char_index)

In [ ]:
# encode words with character encoder
def encodeWord(word, dic=char_index, oov=oov_token):
    return [dic[c] if c in dic else dic[oov] for c in list(word)]

X_train_sc_encoded = np.array([encodeWord(each) for each in X_train_sc])
max_len = max([len(each) for each in X_train_sc_encoded])

print('input shape: ', X_train_sc_encoded.shape)
print('max length in input elements: ', max_len)

In [ ]:
# ADD padding to sample
X_train_sc_pad = pad_sequences(X_train_sc_encoded, maxlen=max_len, padding='post')

# convert to One Hot encoding
X_train_sc_OH = to_categorical(X_train_sc_pad, num_classes=len(char_index))

X_train_sc_OH.shape

## **Implementation**

In [ ]:
class SubjectClassificationNN:
  def __init__(self, X_train, y_train):
    self.X_train = X_train
    self.y_train = y_train

    input = Input(shape=X_train.shape[1:])
    x = LSTM(128, return_sequences=True)(input)
    x = GRU(128, return_sequences=True)(x)

    flatten = Flatten()(x)

    out = Dense(256, activation="tanh")(flatten)
    out = Dense(64, activation="tanh")(out)
    out = Dense(6, activation="softmax")(out)

    # Compile the Model
    self.model = Model(inputs=input, outputs=out)
    self.model.compile(loss='mean_squared_error', optimizer=Adam(0.001), metrics=['accuracy'])
    # self.model.summary()


  def train(self):
    filepath = 'my_best_model.hdf5'
    checkpoint = ModelCheckpoint(filepath=filepath,
                              monitor='accuracy',
                              verbose=0,
                              save_best_only=True,
                              mode='max')
    
    self.history = self.model.fit(self.X_train, self.y_train, epochs=1000, batch_size=64, verbose=0, callbacks=[checkpoint]).history

    self.model = load_model(filepath)

  def getModelResult(self):
    return self.model.evaluate(self.X_train, self.y_train)

  def predictClass(self, subjects):
    encoded_subjects = encodeSubjects(subjects) # encodes input
    pred = self.model.predict(encoded_subjects) # calculate probablity
    pred_OH = argmaxKeepDimensions(pred)  # One hot classification
    pred_class = convertOneHotToClassNumber(pred_OH)  # class number
    return pred_class
      

# **Preprocessing**

---



## **Generate Dataset**

In [ ]:
path = 'Datasets/raw_dataset.xlsx'
# read csv data from github
dataset = pd.read_excel(path)
# delete rows with nan elements
dataset = dataset.dropna()

In [ ]:
dataset.head()

In [ ]:
inputs = np.array(dataset[['sentence1', 'sentence2']])
scores = np.array(dataset['score'])
print(inputs.shape)
print(scores.shape)

In [ ]:
# hazm objects
normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
stemmer = hazm.Stemmer()
tagger = hazm.POSTagger(model='resources/postagger.model')
parser = hazm.DependencyParser(tagger = tagger, lemmatizer = lemmatizer)

In [ ]:
# normalzing data
for i in range(len(inputs)):
  inputs[i] = [normalizer.normalize(inputs[i][0]), normalizer.normalize(inputs[i][1])]

In [ ]:
def findSubject(input):
  tokens = hazm.word_tokenize(input)
  parseGraph = parser.parse(tokens)
  # subjects of each sentence
  subjects = []
  for i in range(len(tokens)):
    tokenDetails = parseGraph.get_by_address(i)
    if tokenDetails['rel'] == 'SBJ':
      subjects.append(tokenDetails['word'])
  return subjects

def findSubjects(input):
  for i in range(len(input)):
    print(i, end=', ', )
    subjects1Col.append(','.join([str(elem) for elem in findSubject(input[i][0])]))
    subjects2Col.append(','.join([str(elem) for elem in findSubject(input[i][1])]))

In [ ]:
subjects1Col = []
subjects2Col = []
with tf.device('/GPU:0'):
  findSubjects(inputs)

dftest = pd.DataFrame(scores, columns=['score'])
dftest['Sentence1'] = inputs[:, 0]
dftest['Subject1'] = subjects1Col
dftest['Sentence2'] = inputs[:, 1]
dftest['Subject2'] = subjects2Col
dftest.style
dftest.to_excel("dataset6.xlsx", sheet_name='Sheet_name_1')

## **Load Generated Dataset**

In [ ]:
dataset = pd.read_excel('Datasets/dataset.xlsx')

In [ ]:
dataset = dataset.replace(to_replace = np.nan, value = 'NA')

In [ ]:
# drop rows with unacceptable scores
for j in range(len(dataset)):
  try:
      dataset['score'][j] = float(dataset['score'][j])
  except (TypeError, ValueError):
      dataset.drop(j, axis=0, inplace=True)


In [ ]:
dataset.head()
dataset.shape

In [ ]:
sub1 = np.array(dataset['Subject1'])
sub2 = np.array(dataset['Subject2'])
score = np.array(dataset['score'])

In [ ]:
# NN for subject classification
with tf.device('/GPU:0'):
  scnn = SubjectClassificationNN(X_train_sc_OH, y_train_sc)
  scnn.train()
  scnn.getModelResult()

In [ ]:
# Subject Classification for samples

subClass = []

with tf.device('/GPU:0'):
  for i in range(len(sub1)):
    sub1_classes = scnn.predictClass(sub1[i].split(','))
    sub2_classes = scnn.predictClass(sub2[i].split(','))

    commonClasses = np.intersect1d(sub1_classes, sub2_classes)
    if commonClasses.size:
      subClass.append(commonClasses[0]) # append common class number between subjects
    else:
      subClass.append(6)  # append NA class(6) for non-uniform subjects
  
subClass = np.array(subClass, dtype='int32')

In [ ]:
# Score Classification
print('class 1: ', np.count_nonzero(score < 2.6))
print('class 2: ', np.count_nonzero(score < 3.4) - np.count_nonzero(score < 2.6))
print('class 3: ', np.count_nonzero(score < 4) - np.count_nonzero(score < 3.4))
print('class 4: ', np.count_nonzero(score < 4.5) - np.count_nonzero(score < 4))
print('class 5: ', np.count_nonzero(score <= 5) - np.count_nonzero(score < 4.5))

scoreClass = []

for x in score:
  if x < 2.6:
    scoreClass.append(1)
  elif x < 3.4:
    scoreClass.append(2)
  elif x < 4:
    scoreClass.append(3)
  elif x < 4.5:
    scoreClass.append(4)
  elif x <= 5:
    scoreClass.append(5)
    
scoreClass = np.array(scoreClass)

print('subject classes shape: ', subClass.shape)
print('score classes shape: ', scoreClass.shape)

In [ ]:
# save subject classes into dataframe and save dataframe
dataset['subject classes'] = subClass
dataset['score classes'] = scoreClass

dataset.to_excel("datasetwithclasses.xlsx")
dataset.style

## **Preprocessing Generated Classification Dataset**

### **Dataset**

In [ ]:
dataset = pd.read_excel('Datasets/datasetwithclasses.xlsx')

In [ ]:
dataset.head()

In [ ]:
subClasses = np.array(dataset['subject classes'])
scoreClasses = np.array(dataset['score classes'])

print("Subject Class Men/Boy: ", np.count_nonzero(subClasses == 1))
print("Subject Class Women/Girl: ", np.count_nonzero(subClasses == 2))
print("Subject Class Child: ", np.count_nonzero(subClasses == 3))
print("Subject Class Animal: ", np.count_nonzero(subClasses == 4))
print("Subject Class Other: ", np.count_nonzero(subClasses == 5))
print("Subject Class NA: ", np.count_nonzero(subClasses == 6))
scoreClasses.shape
#dataset.style

### **Create Y_true**

In [ ]:
subClasses_OH = to_categorical(subClasses, num_classes=7)[:, 1:]
scoreClasses_OH = to_categorical(scoreClasses, num_classes=6)[:, 1:]

print(scoreClasses_OH.shape)
print(subClasses_OH.shape)

In [ ]:
# Peparing Y true 
y = np.concatenate((subClasses_OH, scoreClasses_OH), axis=1)
y.shape

### **Create X**

In [ ]:
# Preparing X
sentence1 = np.array(dataset['Sentence1'])
sentence2 = np.array(dataset['Sentence2'])

In [ ]:
# Keras Tokenizer
filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
num_words = 1000

sentenceTokenizer = Tokenizer(num_words=num_words, filters=filters, split=' ', oov_token='UNK')

# train on sentences
sentenceTokenizer.fit_on_texts(sentence1)
# sentenceTokenizer.fit_on_texts(sentence2)

# encode sentences
sentence1_encoded = sentenceTokenizer.texts_to_sequences(sentence1)
sentence2_encoded = sentenceTokenizer.texts_to_sequences(sentence2)

In [ ]:
# max number of words in a sentence
max_words_num = max(max([len(x) for x in sentence1_encoded]), max([len(x) for x in sentence2_encoded]))
max_words_num

In [ ]:
# normalize sentence lenght with padding
maxlen = 20
sentence1_pad = pad_sequences(sentence1_encoded, maxlen=maxlen, padding='post')
sentence2_pad = pad_sequences(sentence2_encoded, maxlen=maxlen, padding='post')
sentence1_pad.shape, sentence2_pad.shape

In [ ]:
# change input to One Hot
sentence1_OH = to_categorical(sentence1_pad, num_classes=num_words)
sentence2_OH = to_categorical(sentence2_pad, num_classes=num_words)
sentence1_OH.shape, sentence2_OH.shape

In [ ]:
# concatenate two sentences
sentences_OH = np.concatenate((sentence1_OH, sentence2_OH), axis=1)
sentences_OH.shape

### **Split data**

In [ ]:
X_train_1, X_test_1, X_train_2, X_test_2, y_train, y_test = train_test_split(sentence1_OH, sentence2_OH, y, test_size=0.2)
X_train_1.shape, X_train_2.shape, y_train.shape

In [ ]:
X_train_list = [X_train_1, X_train_2]
X_test_list = [X_test_1, X_test_2]

# **Implementation**

---



In [ ]:
class RNN:
  def __init__(self, X_train, y_train, model_num):
    self.X_train = X_train
    self.y_train = y_train

    input1 = Input(shape=np.array(X_train[0]).shape[1:])
    input2 = Input(shape=np.array(X_train[1]).shape[1:])

    if model_num == 1:
      m = self.LSTM_model_1(input1, input2)
    elif model_num == 2:
      m = self.LSTM_model_2(input1, input2)
    elif model_num == 3:
      m = self.LSTM_model_3(input1, input2)
    elif model_num == 4:
      m = self.GRU_model_1(input1, input2)
    elif model_num == 5:
      m = self.GRU_model_2(input1, input2)
    elif model_num == 6:
      m = self.SimpleRNN_model_1(input1, input2)
    elif model_num == 7:
      m = self.SimpleRNN_model_2(input1, input2)
    elif model_num == 8:
      m = self.LSTM_model_4(input1, input2)

    output = Dense(11, activation="sigmoid")(m)

    # Compile the Model
    self.model = Model(inputs=[input1, input2], outputs=output)
    self.model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=['accuracy'])

    # self.model.summary()

  def LSTM_model_1(self, input1, input2):
    x1 = LSTM(128, return_sequences=True)(input1)
    x1 = LSTM(128)(x1)

    x2 = LSTM(128, return_sequences=True)(input2)
    x2 = LSTM(128)(x2)

    c = Concatenate(axis=1)([x1, x2])

    d = Dense(64, activation='relu')(c)
    d = Dense(32, activation='relu')(d)
    d = Dense(16, activation='relu')(d)
    
    return d

  def LSTM_model_2(self, input1, input2):
    x1 = LSTM(128, return_sequences=True)(input1)
    x1 = LSTM(128, return_sequences=True)(x1)
    c1 = GlobalMaxPooling1D()(x1)

    x2 = LSTM(128, return_sequences=True)(input2)
    x2 = LSTM(128, return_sequences=True)(x2)
    c2 = GlobalMaxPooling1D()(x2)

    c = Concatenate(axis=1)([c1, c2])

    d = Dense(64, activation='relu')(c)
    d = Dense(32, activation='relu')(d)
    d = Dense(16, activation='relu')(d)
    
    return d
  
  
  def LSTM_model_3(self, input1, input2):
    x1 = LSTM(128, return_sequences=True)(input1)
    x1 = GRU(128)(x1)
 
    x2 = LSTM(128, return_sequences=True)(input2)
    x2 = GRU(128)(x2)

    c = Concatenate(axis=1)([x1, x2])

    d = Dense(64, activation='relu')(c)
    d = Dense(32, activation='relu')(d)
    d = Dense(16, activation='relu')(d)
    
    return d

  def LSTM_model_4(self, input1, input2):
    x1 = LSTM(128, return_sequences=True)(input1)
    x1 = SimpleRNN(128, return_sequences=True)(x1)
    x1 = SimpleRNN(128, return_sequences=True)(x1)
    c1 = GlobalMaxPooling1D()(x1)

    x2 = LSTM(128, return_sequences=True)(input2)
    x2 = SimpleRNN(128, return_sequences=True)(x2)
    x2 = SimpleRNN(128, return_sequences=True)(x2)
    c2 = GlobalMaxPooling1D()(x2)

    c = Concatenate(axis=1)([c1, c2])

    d = Dense(64, activation='relu')(c)
    d = Dropout(0.2)(d)
    d = Dense(32, activation='relu')(d)
    d = Dropout(0.1)(d)
    d = Dense(16, activation='relu')(d)
    
    return d

  def GRU_model_1(self, input1, input2):
    x1 = GRU(128, return_sequences=True)(input1)
    x1 = GRU(128)(x1)
 
    x2 = GRU(128, return_sequences=True)(input2)
    x2 = GRU(128)(x2)

    c = Concatenate(axis=1)([x1, x2])

    d = Dense(64, activation='relu')(c)
    d = Dense(32, activation='relu')(d)
    d = Dense(16, activation='relu')(d)
    
    return d

  def GRU_model_2(self, input1, input2):
    x1 = GRU(256, return_sequences=True)(input1)
    c1 = GlobalMaxPooling1D()(x1)

    x2 = GRU(256, return_sequences=True)(input2)
    c2 = GlobalMaxPooling1D()(x2)

    c = Concatenate(axis=1)([c1, c2])

    d = Dense(64, activation='relu')(c)
    d = Dense(32, activation='relu')(d)
    d = Dense(16, activation='relu')(d)
    
    return d
  
  def SimpleRNN_model_1(self, input1, input2):
    x1 = SimpleRNN(128, return_sequences=True)(input1)
    x1 = SimpleRNN(128)(x1)

    x2 = SimpleRNN(128, return_sequences=True)(input2)
    x2 = SimpleRNN(128)(x2)

    c = Concatenate(axis=1)([x1, x2])

    d = Dense(64, activation='relu')(c)
    # d = Dense(64, activation='relu')(d)
    d = Dense(16, activation='relu')(d)
    
    return d

  def SimpleRNN_model_2(self, input1, input2):
    x1 = SimpleRNN(128, return_sequences=True)(input1)
    x1 = SimpleRNN(128, return_sequences=True)(x1)
    c1 = GlobalMaxPooling1D()(x1)

    x2 = SimpleRNN(128, return_sequences=True)(input2)
    x2 = SimpleRNN(128, return_sequences=True)(x2)
    c2 = GlobalMaxPooling1D()(x2)

    c = Concatenate(axis=1)([c1, c2])

    d = Dense(64, activation='relu')(c)
    d = Dense(32, activation='relu')(d)
    d = Dense(16, activation='relu')(d)
    
    return d


  def train(self, validation_data):
    filepath = 'my_best_model.hdf5'
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    self.history = self.model.fit(self.X_train, self.y_train, validation_data=validation_data, epochs=50, batch_size=32, verbose=2, callbacks=[checkpoint]).history
    self.model = load_model(filepath)

  def predict(self):
    self.y_train_pre = self.model.predict(self.X_train)
    self.y_test_pre = self.model.predict(self.X_test)
  
  def evaluate(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test
    self.test_result = self.model.evaluate(X_test, y_test, verbose=0)

  def outputResult(self):

    # best models based on acc or loss in tarin set or test set
    trainHistory = list(
        map(lambda x, y: [x, y], self.history['accuracy'], self.history['loss']))
    testHistory = list(
        map(lambda x, y: [x, y], self.history['val_accuracy'], self.history['val_loss']))
    print(
        f"\nbest model based on min training set loss:  acc= {min(trainHistory, key = lambda k: k[1])[0]}  loss= {min(trainHistory, key = lambda k: k[1])[1]}")
    print(
        f"best model based on min test set loss:  acc= {min(testHistory, key = lambda k: k[1])[0]}  loss= {min(testHistory, key = lambda k: k[1])[1]}")
    print(
        f"best model based on max training set accuracy:  acc= {max(trainHistory, key = lambda k: k[0])[0]}  loss= {max(trainHistory, key = lambda k: k[0])[1]}")
    print(
        f"best model based on max test set accuracy:  acc= {max(testHistory, key = lambda k: k[0])[0]}  loss= {max(testHistory, key = lambda k: k[0])[1]}")

    print("\nevaluate dataset with best model based on maximum test set accuracy")
    print("evaluate train set= ", self.model.evaluate(
        self.X_train, self.y_train, verbose=0))
    print("evaluate test set= ", self.test_result)

    y_train_pred = self.model.predict(self.X_train)
    # convert probablities to 0s and 1s
    y_train_pred_score = argmaxKeepDimensions(y_train_pred[:, :6])
    y_train_pred_subject = argmaxKeepDimensions(y_train_pred[:, 6:])
    y_train_pred = np.concatenate((y_train_pred_score, y_train_pred_subject), axis=1)

    y_test_pred = self.model.predict(self.X_test)
    # convert probablities to 0s and 1s
    y_test_pred_score = argmaxKeepDimensions(y_test_pred[:, :6])
    y_test_pred_subject = argmaxKeepDimensions(y_test_pred[:, 6:])
    y_test_pred = np.concatenate((y_test_pred_score, y_test_pred_subject), axis=1)

    # confusion matrix and precision, recall and f1 report
    print('\n', '-'*30, 'metrics for traning set', '-'*30)
    print("confusion matrix: \n", metrics.multilabel_confusion_matrix(self.y_train, y_train_pred))
    print(metrics.classification_report(self.y_train,
          y_train_pred, digits=3, target_names=['Men/Boy', 'Women/Girl', 'Child', 'Animal', 'Other', 'NA', 'Score1', 'Score2', 'Score3', 'Score4', 'Score5']))

    # confusion matrix and precision, recall and f1 report
    print('-'*30, 'metrics for test set', '-'*30)
    print("confusion matrix: \n", metrics.multilabel_confusion_matrix(self.y_test, y_test_pred))
    print(metrics.classification_report(self.y_test,
          y_test_pred, digits=3, target_names=['Men/Boy', 'Women/Girl', 'Child', 'Animal', 'Other', 'NA', 'Score1', 'Score2', 'Score3', 'Score4', 'Score5']))
    
    # self.model.summary()
    
      

# **Simple RNN**

## **Model 1**

In [ ]:
with tf.device('/GPU:0'):
  model6 = RNN(X_train_list, y_train, model_num=6)
  model6.train((X_test_list, y_test))
  model6.evaluate(X_test_list, y_test)
  model6.outputResult()

## **Model 2**

In [ ]:
with tf.device('/GPU:0'):
  model7 = RNN(X_train_list, y_train, model_num=7)
  model7.train((X_test_list, y_test))
  model7.evaluate(X_test_list, y_test)
  model7.outputResult()

In [ ]:
showPlots(model7)

In [ ]:
plot_model(model7.model, to_file='/SimpleRNN_plot.png')

# **GRU Model**

## **Model 1**

In [ ]:
with tf.device('/GPU:0'):
  model4 = RNN(X_train_list, y_train, model_num=4)
  model4.train((X_test_list, y_test))
  model4.evaluate(X_test_list, y_test)
  model4.outputResult()

In [ ]:
showPlots(model4)

In [ ]:
plot_model(model4.model, to_file='/content/GRU_plot.png')

## **Model 2**

In [ ]:
with tf.device('/GPU:0'):
  model5 = RNN(X_train_list, y_train, model_num=5)
  model5.train((X_test_list, y_test))
  model5.evaluate(X_test_list, y_test)
  model5.outputResult()

# **LSTM Models**

## **Model 1**

In [ ]:
with tf.device('/GPU:0'):
  model1 = RNN(X_train_list, y_train, model_num=1)
  model1.train((X_test_list, y_test))
  model1.evaluate(X_test_list, y_test)
  model1.outputResult()

In [ ]:
with tf.device('/GPU:0'):
  model11 = RNN(X_train_list, y_train, model_num=1)
  model11.train((X_test_list, y_test))
  model11.evaluate(X_test_list, y_test)
  model11.outputResult()

## **Model 2**

In [ ]:
with tf.device('/GPU:0'):
  model21 = RNN(X_train_list, y_train, model_num=2)
  model21.train((X_test_list, y_test))
  model21.evaluate(X_test_list, y_test)
  model21.outputResult()

## **Model 3**

In [ ]:
with tf.device('/GPU:0'):
  model3 = RNN(X_train_list, y_train, model_num=3)
  model3.train((X_test_list, y_test))
  model3.evaluate(X_test_list, y_test)
  model3.outputResult()

## **Model 4**

In [ ]:
with tf.device('/GPU:0'):
  model8 = RNN(X_train_list, y_train, model_num=8)
  model8.train((X_test_list, y_test))
  model8.evaluate(X_test_list, y_test)
  model8.outputResult()

In [ ]:
with tf.device('/GPU:0'):
  model81 = RNN(X_train_list, y_train, model_num=8)
  model81.train((X_test_list, y_test))
  model81.evaluate(X_test_list, y_test)
  model81.outputResult()

In [ ]:
showPlots(model81)

In [ ]:
plot_model(model8.model, to_file='/content/LSTM_plot.png')